In [ ]:
!pip install -U bitsandbytes

In [ ]:
pip install transformers

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
from utils import *

In [ ]:
token = "YourTokenHere"

from huggingface_hub import login

login(token)

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf" #Running on model LLaMA 2-7B

tokenizer = AutoTokenizer.from_pretrained(model_name, token = token)

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.unk_token

model1 = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config,
    device_map="auto", token=token, torch_dtype=torch.float16)
device = 'cuda:0' #Change according to device
model1.to(device)

In [ ]:
path_h = "/content/datasets/llama_harmful_strings.csv"
path_b = "/content/datasets/llama_benign.csv"
harmful_dataset = pd.read_csv(path_h)
benign_dataset = pd.read_csv(path_b)

In [ ]:
print(harmful_dataset.columns)
print(benign_dataset.columns)

In [ ]:
harmful_inst = harmful_dataset['instruction'].tolist()
harmful_output = harmful_dataset['output'].tolist()
benign_inst = benign_dataset['instruction'].tolist()
benign_output = benign_dataset['output'].tolist()

In [ ]:
blank_space = tokenizer.encode(" ", add_special_tokens=False)
new_line = tokenizer.encode("\n", add_special_tokens=False)

In [ ]:
setup(tokenizer)

In [ ]:
b_stats = []
with open("benign_stats.txt", "w") as f:
    f.write(f"Benign Stats \n")
for i in range(len(benign_output)):
  print(str(i) + " Start")
  start = time.perf_counter()
  gen(model1, tokenizer,benign_output[i], b_stats)
  end = time.perf_counter()
  elapsed = end - start
  print(f"Sample took {elapsed:.6f} seconds")
  with open("benign_stats.txt", "a") as f:
    f.write(f"{[b_stats[-1], elapsed]}\n")
  print(str(i) + " End")

In [ ]:
h_stats = []
with open("harmful_stats.txt", "w") as f:
    f.write(f"Harmful Stats \n")
for i in range(len(harmful_output)):
  print(str(i) + " Start")
  start = time.perf_counter()
  gen(model1, tokenizer, harmful_output[i], h_stats)
  end = time.perf_counter()
  elapsed = end - start
  print(f"Sample took {elapsed:.6f} seconds")
  with open("harmful_stats.txt", "a") as f:
    f.write(f"{[h_stats[-1], elapsed]}\n")
  print(str(i) + " End")

In [ ]:
c = 0
for x in h_stats:
    if x[3]==False:
        c+=1

In [ ]:
print("False Negatives: " + str(c))

In [ ]:
b = 0
for x in b_stats:
    if x[3]==True:
        b+=1

In [ ]:
print("False Positives: " + str(b))